In [ ]:
import numpy as np
from collections import Counter

def calculate_frequencies(arr):
    flattened = arr.flatten()
    return Counter(flattened)

# Example usage
arr = np.array([[1, 2, 3], [1, 1, 2]])
frequencies = calculate_frequencies(arr)
print(frequencies)


In [ ]:
import heapq

class HuffmanNode:
    def __init__(self, value=None, freq=0):
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(frequencies):
    heap = [HuffmanNode(value, freq) for value, freq in frequencies.items()]
    heapq.heapify(heap)
    
    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)
        merged = HuffmanNode(freq=node1.freq + node2.freq)
        merged.left = node1
        merged.right = node2
        heapq.heappush(heap, merged)
    
    return heap[0]

# Example usage
huffman_tree = build_huffman_tree(frequencies)


In [ ]:
def generate_huffman_codes(node, prefix='', codebook=None):
    if codebook is None:
        codebook = {}
    if node.value is not None:
        codebook[node.value] = prefix
    else:
        generate_huffman_codes(node.left, prefix + '0', codebook)
        generate_huffman_codes(node.right, prefix + '1', codebook)
    return codebook

# Example usage
codebook = generate_huffman_codes(huffman_tree)
print(codebook)


In [ ]:
def encode_array(arr, codebook):
    flattened = arr.flatten()
    encoded_bits = ''.join(codebook[value] for value in flattened)
    return encoded_bits

# Example usage
encoded_bits = encode_array(arr, codebook)
print(encoded_bits)


In [ ]:
import struct
import json

def save_encoded_data(encoded_bits, codebook, original_shape, file_path):
    with open(file_path, 'wb') as f:
        # Save the original shape
        f.write(struct.pack('>II', *original_shape))
        
        # Save the codebook
        codebook_str = json.dumps(codebook)
        f.write(struct.pack('>I', len(codebook_str)))
        f.write(codebook_str.encode('utf-8'))
        
        # Save the encoded bitstream length
        bitstream_len = len(encoded_bits)
        f.write(struct.pack('>I', bitstream_len))
        
        # Save the encoded bitstream
        byte_array = bytearray()
        for i in range(0, bitstream_len, 8):
            byte = encoded_bits[i:i+8]
            byte_array.append(int(byte.ljust(8, '0'), 2))
        f.write(byte_array)

# Example usage
save_encoded_data(encoded_bits, codebook, arr.shape, 'encoded_image.bin')


In [ ]:
def load_encoded_data(file_path):
    with open(file_path, 'rb') as f:
        # Read the original shape
        original_shape = struct.unpack('>II', f.read(8))
        
        # Read the codebook
        codebook_len = struct.unpack('>I', f.read(4))[0]
        codebook_str = f.read(codebook_len).decode('utf-8')
        codebook = json.loads(codebook_str)
        
        # Read the encoded bitstream length
        bitstream_len = struct.unpack('>I', f.read(4))[0]
        
        # Read the encoded bitstream
        bitstream_bytes = f.read()
        encoded_bits = ''.join(format(byte, '08b') for byte in bitstream_bytes)[:bitstream_len]
    
    return encoded_bits, codebook, original_shape

def decode_huffman(encoded_bits, codebook):
    reverse_codebook = {v: k for k, v in codebook.items()}
    decoded_values = []
    
    current_bits = ''
    for bit in encoded_bits:
        current_bits += bit
        if current_bits in reverse_codebook:
            decoded_values.append(reverse_codebook[current_bits])
            current_bits = ''
    
    return decoded_values

def bits_to_array(decoded_values, original_shape):
    return np.array(decoded_values).reshape(original_shape)

# Example usage
encoded_bits, codebook, original_shape = load_encoded_data('encoded_image.bin')
decoded_values = decode_huffman(encoded_bits, codebook)
reconstructed_array = bits_to_array(decoded_values, original_shape)
print(reconstructed_array)